In [ ]:
# Import the required modules and libraries:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

In [ ]:
# Resize the size of all images to a unanimous value (224, 224).
# Convert PIL image objects into Tensors.
# Normalize the tensor values based on the mean and standard deviation of the RGB values of all the images:


data_transforms = transforms.Compose([
transforms.Resize((224,224)),
transforms.RandomHorizontalFlip(),
transforms.ToTensor(),
transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
# Create an object of torchvision.datasets.CIFAR100 to get the training and testing set:

trainset = datasets.CIFAR100(root = "/content/drive/MyDrive/5_Cisc442_CompVision/PR2", train = True, transform = data_transforms, download=True)
testset = datasets.CIFAR100(root = "/content/drive/MyDrive/5_Cisc442_CompVision/PR2", train=False, transform = data_transforms, download=True)

100%|██████████| 169001437/169001437 [00:01<00:00, 90571355.93it/s]


Extracting /content/drive/MyDrive/5_Cisc442_CompVision/PR2/cifar-100-python.tar.gz to /content/drive/MyDrive/5_Cisc442_CompVision/PR2
Files already downloaded and verified


In [ ]:
# Create a data loader.

data = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
dataTestSet = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=True)

In [ ]:
# Load a VGG16 network with pretrained ImageNet weights:

model = models.vgg16(pretrained = True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:04<00:00, 120MB/s]


In [ ]:
# Extract the number of input features for the last fully connected layer of the model:

num_in_ftrs = model.classifier[6].in_features

In [ ]:
# Replace the last fully connected layer with a new layer.
# The new layer has the same number of input features as the
# original network but the number of outputs should be
# equal to the number of classes in the CIFAR100 dataset.

num_cls = len(trainset.classes) # number of classes in the CIFAR100 dataset
model.classifier[6] = nn.Linear(num_in_ftrs, num_cls) # num_cls is the number of classes.

In [ ]:
# We are using pretrained weights from the ImageNet dataset.
# The last layer of VGG16 has been replaced for fitting with our dataset (CIFAR100).
# Except for the new last layer, weights from other layers need to be frozen.
# It means that these weights will not be updated during the training.

for param in model.parameters(): # freeze all the layers
  param.requires_grad = False
for param in model.classifier[6].parameters(): # unfreeze the last linear layer.
  param.requires_grad = True

In [ ]:
# Set the number of epochs:
num_epochs = 10

In [ ]:
# Move the model to GPU (if available):
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
# Define a loss function for evaluating the trained model:
criterion = nn.CrossEntropyLoss()

In [ ]:
# Create an optimizer with an initial learning rate and momentum:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Create a scheduler to control the way that learning rate changes during the training process:
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
'''
Iterate over the epochs and save the best model weights.
Basically, the best model can achieve the best accuracy during the iteration.
In every iteration, we get a mini-batch of images and their corresponding labels.
Use zero_grad() to reset the calculated gradients.
Use the current model weights for predication and backpropagate the prediction loss.
After iterating over all batches and if we are in the training phase, we need to run scheduler.step() to update the scheduler status.

So, for each batch, the pseudo-code is:
images, labels = data # data is a mini-batch input
optimizer.zero_grad()
outputs = model(images) # here the model is the pretrained VGG16
loss = criterion(outputs, labels)
loss.backward()
optimizer.step()
'''

model.train()

# Iterate over the epochs and save the best model weights.
for i in range(num_epochs):
  for batch in data:

    # In every iteration, we get a mini-batch of images and their corresponding labels.
    # data is a mini-batch input
    images, labels = batch # data is a mini-batch input
    images = images.to(device)
    labels = labels.to(device)

    # Use zero_grad() to reset the calculated gradients.
    optimizer.zero_grad()

    # Use the current model weights for predication and backpropagate the prediction loss.
    outputs = model(images) # here the model is the pretrained VGG16
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

  # After iterating over all batches and if we are in the training phase, we need to run scheduler.step() to update the scheduler status.
  scheduler.step()




In [ ]:
# Save the model weights:
torch.save(model.state_dict(), 'best_model.pth')

In [ ]:
# The testing process is very similar to the training process except that there is no need to backpropagate the loss.
# For testing the model, first, you need to prepare the model in the same way that we prepared it for the
# training process and load the best model that we saved in the training process.
model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

In [ ]:
'''
After loading the model weights, set the model to evaluation mode.
Then go through the test set, and predict the category of images,
and compute the number of correctly classified images and the accuracy.

So, for each batch, the pseudo-code is:
images, labels = data
outputs = model(images)
_, preds = torch.max(outputs, 1)
'''
total = 0
trues = 0

# After loading the model weights, set the model to evaluation mode.
model.eval()

# Then go through the test set
for batch in dataTestSet:
  images, labels = batch
  images = images.to(device)
  labels = labels.to(device)

  outputs = model(images)

  # predict the category of images
  _, preds = torch.max(outputs, 1)

  # compute the number of correctly classified images and the accuracy
  # correctClassImgs += (outputs == labels).sum().item()
  x = torch.eq(labels, preds)
  for a in range(len(x)):
    total += 1
    if x[a] == True:
      trues += 1


# accuracy = correct / total
accuracy = (trues / total)*100

# Finally, print the accuracy.
print(accuracy)




58.96
